<a href="https://colab.research.google.com/github/ilievskaAna/Employee-Attrition-Prediction/blob/main/Employee_Attrition_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [97]:
import pandas as pd

df = pd.read_csv("/content/employee_attrition_dataset.csv")
print(df.head())

   Employee_ID  Age  Gender Marital_Status Department   Job_Role  Job_Level  \
0            1   58  Female        Married         IT    Manager          1   
1            2   48  Female        Married      Sales  Assistant          5   
2            3   34    Male        Married  Marketing  Assistant          1   
3            4   27  Female       Divorced  Marketing    Manager          1   
4            5   40    Male       Divorced  Marketing  Executive          1   

   Monthly_Income  Hourly_Rate  Years_at_Company  ...  Overtime  \
0           15488           28                15  ...        No   
1           13079           28                 6  ...       Yes   
2           13744           24                24  ...       Yes   
3            6809           26                10  ...        No   
4           10206           52                29  ...        No   

   Project_Count  Average_Hours_Worked_Per_Week  Absenteeism  \
0              6                             54           

In [98]:
df = df.drop(columns=['Employee_ID'])

In [99]:
from sklearn.preprocessing import LabelEncoder

categorical_cols = df.select_dtypes(include=['object']).columns
print('Categorical Columns : ', categorical_cols.tolist())

for col in categorical_cols: df[col] = LabelEncoder().fit_transform(df[col])

Categorical Columns :  ['Gender', 'Marital_Status', 'Department', 'Job_Role', 'Overtime', 'Attrition']


In [100]:
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE

X = df.drop('Attrition', axis=1)
y = df['Attrition']

sm = SMOTE(random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42, stratify=y)
X_train_res, y_train_res = sm.fit_resample(X_train, y_train)

In [101]:
from xgboost import XGBClassifier

xgb = XGBClassifier(scale_pos_weight=4.32, random_state=42)
xgb.fit(X_train_res, y_train_res)
y_pred = xgb.predict(X_test)

In [102]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

y_pred = xgb.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))

Accuracy: 0.7
              precision    recall  f1-score   support

           0       0.83      0.80      0.81       203
           1       0.24      0.28      0.26        47

    accuracy                           0.70       250
   macro avg       0.53      0.54      0.53       250
weighted avg       0.72      0.70      0.71       250


Confusion Matrix:
 [[162  41]
 [ 34  13]]
